In [142]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.model_selection as ms
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error 
from sklearn.preprocessing import StandardScaler

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

# Data preprocessing

<ul>
    <b>Removing columns</b>:
    <li>id, host_id - ids</li>
    <li>name - name of listing (offer)</li>
    <li>host_name - name of user hosting a listing</li>
    <li>latitude, longitude - we're going to use neighbourhood and neighberhood group as geogrephical regions rather then exact latitude ang longitude </li>
    <li>last_review - contains a lot of nulls which are really tough to replace</li>
    <li>calculated_host_listings_count - amount of listings hosted by one user</li>
</ul>

<br><br>

Replacing column names to make it easier to manipualate
<ul>
<b>Replaced names</b>
    <li>neighbourhood_group -> nghb_group</li>
    <li>neighbourhood -> nghb </li>
</ul>

In [236]:
df = pd.read_csv('AB_NYC_2019.csv')
df_org = df

cols_to_drop = ['id', 'name', 'host_id', 'host_name', 
                'last_review', 'calculated_host_listings_count']
df = df.drop(axis=1, columns=cols_to_drop)

cols_replace = {'neighbourhood_group': 'nghb_group',
                'neighbourhood': 'nghb'}
df = df.rename(mapper=cols_replace, axis=1)

<b>Removing NaN</b><br>
Replacing NaN values from reviews_per_month with 0, as NaN is effectt of no reviews. There are no other NaN values in any other columns.

In [237]:
df[np.isnan(df.reviews_per_month)]

,nghb_group,nghb,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,availability_365
2,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,365
19,Manhattan,East Harlem,40.79685,-73.94872,Entire home/apt,190,7,0,NaN,249
26,Manhattan,Inwood,40.86754,-73.92639,Private room,80,4,0,NaN,0
36,Brooklyn,Bedford-Stuyvesant,40.68876,-73.94312,Private room,35,60,0,NaN,365
38,Brooklyn,Flatbush,40.63702,-73.96327,Private room,150,1,0,NaN,365
...,...,...,...,...,...,...,...,...,...,...
48890,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,NaN,9
48891,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,NaN,36
48892,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,NaN,27
48893,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,NaN,2


In [238]:
df[df.reviews_per_month == 0]

,nghb_group,nghb,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,availability_365


In [239]:
df['reviews_per_month'] = df['reviews_per_month'].fillna(0)
df = df.drop(df.index[df.price==0])

df.isna().sum()

nghb_group           0
nghb                 0
latitude             0
longitude            0
room_type            0
price                0
minimum_nights       0
number_of_reviews    0
reviews_per_month    0
availability_365     0
dtype: int64

Dropping rows with prices bigger than 500, as AirBnB is intended for rather cheap and one night rent. Higher prices will significantly decrease r^2 (from about 0.5 to 0.18). Furthermore, when looking at value counts for more than 500 USD it is only slightly more than 400-500 range.

In [240]:
bins = np.arange(0, 500, 100)
bins = np.append(bins, [500, 1000, 10000])
df_org['price'].value_counts(bins=bins).sort_index()

(-0.001, 100.0]      23928
(100.0, 200.0]       16583
(200.0, 300.0]        5027
(300.0, 400.0]        1594
(400.0, 500.0]         719
(500.0, 1000.0]        805
(1000.0, 10000.0]      239
Name: price, dtype: int64

In [241]:
df = df.drop(df[df.price > 300].index, axis=0)

In [ ]:
pd.DataFrame(df.corr()["price"])

Factorizing categorical values, so that there are no strings but rather each category has assigned value from 0...n, where n is number of unique categories.

In [242]:
df['nghb_group'] = pd.factorize(df['nghb_group'])[0]
df['nghb'] = pd.factorize(df['nghb'])[0]
df['room_type'] = pd.factorize(df['room_type'])[0]

Dividing dataset into train and test

In [243]:
X = df.drop("price", axis=1)
y = df["price"]

scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=df.drop("price", axis=1).columns)

X_train, X_test, y_train, y_test = ms.train_test_split(X, y, random_state=42)
# no stratify as it is regression problem not classification 

In [248]:
model_v1 = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=X_train.shape[1]),
    keras.layers.Dense(500, activation="relu"),
    keras.layers.Dense(250, activation="relu"),
    keras.layers.Dense(125, activation="relu"),
    keras.layers.Dense(62, activation="relu"),
    keras.layers.Dense(1),
])

model_v1.compile(loss='mean_absolute_error',
             optimizer='adam',
             metrics=['MeanSquaredError',
                      'RootMeanSquaredError',
                      'MeanAbsoluteError'])
    
model_v1.summary()



model_v2 = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=X_train.shape[1]),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(1),
])

model_v2.compile(loss='mean_absolute_error',
             optimizer='adam',
             metrics=['MeanSquaredError',
                      'RootMeanSquaredError',
                      'MeanAbsoluteError'])

model_v2.summary()

Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_120 (Dense)            (None, 500)               5000      
_________________________________________________________________
dense_121 (Dense)            (None, 250)               125250    
_________________________________________________________________
dense_122 (Dense)            (None, 125)               31375     
_________________________________________________________________
dense_123 (Dense)            (None, 62)                7812      
_________________________________________________________________
dense_124 (Dense)            (None, 1)                 63        
Total params: 169,500
Trainable params: 169,500
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_36"
_________________________________________________________________
Layer (type)            

Using normalizer which in practice does the standarization of all values.

In [250]:
history_v1 = model_v1.fit(X_train, y_train, epochs=100)

Epoch 1/100
1068/1068 [==============================] - 4s 4ms/step - loss: 32.3221 - mean_squared_error: 2125.4673 - root_mean_squared_error: 46.1028 - mean_absolute_error: 32.3221
Epoch 2/100
1068/1068 [==============================] - 4s 4ms/step - loss: 32.2691 - mean_squared_error: 2116.1399 - root_mean_squared_error: 46.0015 - mean_absolute_error: 32.2691
Epoch 3/100
1068/1068 [==============================] - 4s 4ms/step - loss: 32.0241 - mean_squared_error: 2104.8611 - root_mean_squared_error: 45.8788 - mean_absolute_error: 32.0241
Epoch 4/100
1068/1068 [==============================] - 4s 4ms/step - loss: 31.8925 - mean_squared_error: 2085.0430 - root_mean_squared_error: 45.6623 - mean_absolute_error: 31.8925
Epoch 5/100
1068/1068 [==============================] - 4s 4ms/step - loss: 31.7630 - mean_squared_error: 2079.8992 - root_mean_squared_error: 45.6059 - mean_absolute_error: 31.7630
Epoch 6/100
1068/1068 [==============================] - 4s 4ms/step - loss: 31.6547 

1068/1068 [==============================] - 4s 4ms/step - loss: 30.2504 - mean_squared_error: 1934.4427 - root_mean_squared_error: 43.9823 - mean_absolute_error: 30.2504
Epoch 45/100
1068/1068 [==============================] - 4s 4ms/step - loss: 30.1597 - mean_squared_error: 1926.9484 - root_mean_squared_error: 43.8970 - mean_absolute_error: 30.1597
Epoch 46/100
1068/1068 [==============================] - 4s 4ms/step - loss: 30.1726 - mean_squared_error: 1923.2953 - root_mean_squared_error: 43.8554 - mean_absolute_error: 30.1726
Epoch 47/100
1068/1068 [==============================] - 4s 4ms/step - loss: 30.0952 - mean_squared_error: 1915.9385 - root_mean_squared_error: 43.7714 - mean_absolute_error: 30.0952
Epoch 48/100
1068/1068 [==============================] - 4s 4ms/step - loss: 30.0685 - mean_squared_error: 1914.6572 - root_mean_squared_error: 43.7568 - mean_absolute_error: 30.0685
Epoch 49/100
1068/1068 [==============================] - 4s 4ms/step - loss: 30.0848 - mean_

1068/1068 [==============================] - 4s 4ms/step - loss: 28.8681 - mean_squared_error: 1814.9930 - root_mean_squared_error: 42.6027 - mean_absolute_error: 28.8681
Epoch 89/100
1068/1068 [==============================] - 4s 4ms/step - loss: 28.8326 - mean_squared_error: 1807.7205 - root_mean_squared_error: 42.5173 - mean_absolute_error: 28.8326
Epoch 90/100
1068/1068 [==============================] - 5s 4ms/step - loss: 28.7868 - mean_squared_error: 1802.7841 - root_mean_squared_error: 42.4592 - mean_absolute_error: 28.7868
Epoch 91/100
1068/1068 [==============================] - 5s 5ms/step - loss: 28.7077 - mean_squared_error: 1797.9960 - root_mean_squared_error: 42.4028 - mean_absolute_error: 28.7077
Epoch 92/100
1068/1068 [==============================] - 4s 4ms/step - loss: 28.6680 - mean_squared_error: 1790.4270 - root_mean_squared_error: 42.3134 - mean_absolute_error: 28.6680
Epoch 93/100
1068/1068 [==============================] - 5s 5ms/step - loss: 28.6389 - mean_

In [251]:
history_v2 = model_v2.fit(X_train, y_train, epochs=100)

Epoch 1/100
1068/1068 [==============================] - 4s 3ms/step - loss: 100.2056 - mean_squared_error: 14739.5608 - root_mean_squared_error: 120.8955 - mean_absolute_error: 100.2056
Epoch 2/100
1068/1068 [==============================] - 3s 3ms/step - loss: 40.8756 - mean_squared_error: 3259.5072 - root_mean_squared_error: 57.0889 - mean_absolute_error: 40.8756
Epoch 3/100
1068/1068 [==============================] - 3s 3ms/step - loss: 37.9981 - mean_squared_error: 2955.0610 - root_mean_squared_error: 54.3546 - mean_absolute_error: 37.9981
Epoch 4/100
1068/1068 [==============================] - 3s 3ms/step - loss: 36.9342 - mean_squared_error: 2858.8835 - root_mean_squared_error: 53.4655 - mean_absolute_error: 36.9342
Epoch 5/100
1068/1068 [==============================] - 3s 3ms/step - loss: 36.5565 - mean_squared_error: 2836.9959 - root_mean_squared_error: 53.2607 - mean_absolute_error: 36.5565
Epoch 6/100
1068/1068 [==============================] - 4s 3ms/step - loss: 36.1

1068/1068 [==============================] - 4s 4ms/step - loss: 31.5974 - mean_squared_error: 2041.1708 - root_mean_squared_error: 45.1763 - mean_absolute_error: 31.5974
Epoch 46/100
1068/1068 [==============================] - 4s 3ms/step - loss: 31.9175 - mean_squared_error: 2095.1578 - root_mean_squared_error: 45.7703 - mean_absolute_error: 31.9175
Epoch 47/100
1068/1068 [==============================] - 4s 3ms/step - loss: 32.0697 - mean_squared_error: 2122.7320 - root_mean_squared_error: 46.0605 - mean_absolute_error: 32.0697
Epoch 48/100
1068/1068 [==============================] - 4s 4ms/step - loss: 31.6828 - mean_squared_error: 2046.5992 - root_mean_squared_error: 45.2354 - mean_absolute_error: 31.6828
Epoch 49/100
1068/1068 [==============================] - 3s 3ms/step - loss: 32.0001 - mean_squared_error: 2116.2897 - root_mean_squared_error: 45.9983 - mean_absolute_error: 32.0001
Epoch 50/100
1068/1068 [==============================] - 4s 3ms/step - loss: 31.8599 - mean_

1068/1068 [==============================] - 4s 3ms/step - loss: 31.7417 - mean_squared_error: 2090.8969 - root_mean_squared_error: 45.7161 - mean_absolute_error: 31.7417
Epoch 89/100
1068/1068 [==============================] - 3s 3ms/step - loss: 31.3685 - mean_squared_error: 2029.6026 - root_mean_squared_error: 45.0475 - mean_absolute_error: 31.3685
Epoch 90/100
1068/1068 [==============================] - 3s 3ms/step - loss: 31.4836 - mean_squared_error: 2055.2739 - root_mean_squared_error: 45.3339 - mean_absolute_error: 31.4836
Epoch 91/100
1068/1068 [==============================] - 3s 3ms/step - loss: 31.4588 - mean_squared_error: 2043.6058 - root_mean_squared_error: 45.2054 - mean_absolute_error: 31.4588
Epoch 92/100
1068/1068 [==============================] - 3s 3ms/step - loss: 31.6251 - mean_squared_error: 2078.1196 - root_mean_squared_error: 45.5793 - mean_absolute_error: 31.6251
Epoch 93/100
1068/1068 [==============================] - 4s 3ms/step - loss: 31.3777 - mean_

In [253]:
metrics = {}
metrics_v1_test = model_v1.evaluate(X_test, y_test)
y_pred_v1_test = model_v1.predict(X_test)

metrics_v1_train = model_v1.evaluate(X_train, y_train)
y_pred_v1_train = model_v1.predict(X_train)

metrics_v2_test = model_v2.evaluate(X_test, y_test)
y_pred_v2_test = model_v2.predict(X_test)

metrics_v2_train = model_v2.evaluate(X_train, y_train)
y_pred_v2_train = model_v2.predict(X_train)

metrics["Nr. architektury"] = ["1", "1", "2", "2"]
metrics["Typ danych"] = ["trenujace", "testowe", "trenujace", "testowe"]
metrics["MSE"] = [metrics_v1_train[1], metrics_v1_test[1], metrics_v2_train[1], metrics_v2_test[1]]
metrics["RMSE"] = [metrics_v1_train[2], metrics_v1_test[2], metrics_v2_train[2], metrics_v2_test[2]]
metrics["MAE"] = [metrics_v1_train[3], metrics_v1_test[3], metrics_v2_train[3], metrics_v2_test[3]]
metrics["R2"] = [r2_score(y_train,y_pred_v1_train),
                 r2_score(y_test,y_pred_v1_test),
                 r2_score(y_train,y_pred_v2_train),
                 r2_score(y_test,y_pred_v2_test)]


pd.DataFrame(metrics)

1068/1068 [==============================] - 2s 2ms/step - loss: 31.3655 - mean_squared_error: 2030.7838 - root_mean_squared_error: 45.0642 - mean_absolute_error: 31.3655


,Nr. architektury,Typ danych,MSE,RMSE,MAE,R2
0,1,trenujace,1703.270142,41.270695,27.996891,0.606598
1,1,testowe,2000.362671,44.725414,31.458178,0.542939
2,2,trenujace,2030.783813,45.064217,31.365456,0.530952
3,2,testowe,2041.399048,45.181843,31.603842,0.533563
